In [1]:
import os, sys
sys.path.append(os.path.abspath('../../../main/python'))
import thalesians.tsa.q.qutils as qutils

In [2]:
import datetime as dt
import pandas as pd

In [3]:
from qpython import qconnection

In [4]:
df = pd.DataFrame({
    'sym': ['foo', None, 'baz', 'qux', 'quux', 'quuz', 'corge'],
    'tally': [3, 7, 5, 3, 8, 10, 12],
    'price': [3.57, 10.87, 10.28, 3.22, 18.15, 29.29, 10.09],
    'date': [
        dt.date(2019, 5, 17), dt.date(2019, 5, 17), dt.date(2019, 5, 17),
        dt.date(2019, 5, 17), dt.date(2019, 5, 17), dt.date(2019, 5, 17),
        dt.date(2019, 5, 17)],
    'time': [
        dt.time(21, 43, 54, 357000), dt.time(21, 43, 54, 357000), dt.time(21, 43, 54, 357000),
        dt.time(21, 43, 54, 357000), dt.time(21, 43, 54, 357000), dt.time(21, 43, 54, 357000),
        dt.time(21, 43, 54, 357000)],
    'datetime': [
        dt.datetime(2019, 5, 17, 21, 43, 54, 357000), dt.datetime(2019, 5, 17, 21, 43, 54, 357000),
        dt.datetime(2019, 5, 17, 21, 43, 54, 357000), dt.datetime(2019, 5, 17, 21, 43, 54, 357000),
        dt.datetime(2019, 5, 17, 21, 43, 54, 357000), dt.datetime(2019, 5, 17, 21, 43, 54, 357000),
        dt.datetime(2019, 5, 17, 21, 43, 54, 357000)]
    })

In [5]:
df

,date,datetime,price,sym,tally,time
0,2019-05-17,2019-05-17 21:43:54.357,3.57,foo,3,21:43:54.357000
1,2019-05-17,2019-05-17 21:43:54.357,10.87,None,7,21:43:54.357000
2,2019-05-17,2019-05-17 21:43:54.357,10.28,baz,5,21:43:54.357000
3,2019-05-17,2019-05-17 21:43:54.357,3.22,qux,3,21:43:54.357000
4,2019-05-17,2019-05-17 21:43:54.357,18.15,quux,8,21:43:54.357000
5,2019-05-17,2019-05-17 21:43:54.357,29.29,quuz,10,21:43:54.357000
6,2019-05-17,2019-05-17 21:43:54.357,10.09,corge,12,21:43:54.357000


In [6]:
df.dtypes

date                object
datetime    datetime64[ns]
price              float64
sym                 object
tally                int64
time                object
dtype: object

In [7]:
dtype_to_qtype = {
    'object': None,
    'int64': qutils.QTypes.LONG,
    'float64': qutils.QTypes.FLOAT,
    'bool': qutils.QTypes.BOOLEAN,
    'datetime64[ns]': qutils.QTypes.DATETIME,
    'timedelta[ns]': qutils.QTypes.TIME,
    'category': qutils.QTypes.SYMBOL
}

In [8]:
def infer_q_type(df, column, string_columns=set()):
    typ = dtype_to_qtype[df.dtypes[column].name]
    if typ is None:
        if all([isinstance(x, str) for x in df[column].values if x is not None]):
            typ = qutils.QTypes.SYMBOL
    if typ == qutils.QTypes.SYMBOL and column in string_columns:
        typ = qutils.QTypes.UNTYPED_LIST
    return typ

In [9]:
def df_to_q_table_schema(df, name, string_columns=set()):
    builder = qutils.QCreateTableStatementBuilder(overwrite=True)
    builder.set_table(name)
    for column in df.columns:
        typ = infer_q_type(df, column, string_columns)
        builder.append_column(column, typ, key=False)
    return builder.to_string()

In [10]:
q_code = []

In [11]:
q_code.append(df_to_q_table_schema(df, 'sample'))

In [12]:
def df_to_upsert_statements(df, name, string_columns=set(), q_types=None):
    statements = []
    if q_types is None:
        q_types = {}
        for column in df.columns:
            q_types[column] = infer_q_type(df, column, string_columns)
    for index, row in df.iterrows():
        builder = qutils.QUpsertStatementBuilder()
        builder.set_table(name)
        for column in df.columns:
            builder.append(qutils.make_q_value(row[column], q_types[column]))
        statements.append(builder.to_string())
    return statements

In [13]:
q_code.extend(df_to_upsert_statements(df, 'sample'))

In [14]:
q = qconnection.QConnection(host='localhost', port=2507, pandas=True)
q.open()

In [15]:
for q_line in q_code:
    q(q_line)

In [16]:
q('select from sample')

,date,datetime,price,sym,tally,time
0,2019-05-17,2019-05-17 21:43:54.357,3.57,b'foo',3,21:43:54.357000
1,2019-05-17,2019-05-17 21:43:54.357,10.87,b'',7,21:43:54.357000
2,2019-05-17,2019-05-17 21:43:54.357,10.28,b'baz',5,21:43:54.357000
3,2019-05-17,2019-05-17 21:43:54.357,3.22,b'qux',3,21:43:54.357000
4,2019-05-17,2019-05-17 21:43:54.357,18.15,b'quux',8,21:43:54.357000
5,2019-05-17,2019-05-17 21:43:54.357,29.29,b'quuz',10,21:43:54.357000
6,2019-05-17,2019-05-17 21:43:54.357,10.09,b'corge',12,21:43:54.357000


In [17]:
def df_to_batch_append_statements(df, name, string_columns=set(), q_types=None, rows_per_batch=100):
    builder = qutils.QBatchAppendStatementBuilder(rows_per_batch=rows_per_batch)
    builder.set_table(name)
    if q_types is None:
        q_types = {}
        for column in df.columns:
            q_types[column] = infer_q_type(df, column, string_columns)
    for index, row in df.iterrows():
        builder.start_new_row()
        for column in df.columns:
            builder.append(qutils.make_q_value(row[column], q_types[column]))
    return builder.to_list()

In [18]:
batches = df_to_batch_append_statements(df, 'sample', rows_per_batch=3)

In [19]:
len(batches)

3

In [20]:
batches[0]

'.[(`$"sample");();,;((2019.05.17;2019.05.17T21:43:54.357;3.57f;(`$"foo");3j;21:43:54.357);(2019.05.17;2019.05.17T21:43:54.357;10.87f;`;7j;21:43:54.357);(2019.05.17;2019.05.17T21:43:54.357;10.28f;(`$"baz");5j;21:43:54.357))]'

In [21]:
batches[1]

'.[(`$"sample");();,;((2019.05.17;2019.05.17T21:43:54.357;3.22f;(`$"qux");3j;21:43:54.357);(2019.05.17;2019.05.17T21:43:54.357;18.15f;(`$"quux");8j;21:43:54.357);(2019.05.17;2019.05.17T21:43:54.357;29.29f;(`$"quuz");10j;21:43:54.357))]'

In [22]:
batches[2]

'.[(`$"sample");();,;((2019.05.17;2019.05.17T21:43:54.357;10.09f;(`$"corge");12j;21:43:54.357))]'

In [23]:
q(df_to_q_table_schema(df, 'sample'))

In [24]:
for batch in batches:
    q(batch)